# MCP Integration with AI Assistants

This notebook demonstrates how to use the OChem Helper MCP server for AI-powered chemistry assistance.

In [ ]:
# Setup
import sys
sys.path.append('../src')
sys.path.append('../mcp')

import asyncio
import json
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from IPython.display import display, Markdown

# Import MCP server
from server.ochem_mcp import OChemMCPServer

# Set up async environment
import nest_asyncio
nest_asyncio.apply()

%matplotlib inline

## 1. Initialize MCP Server

Set up the MCP server and explore available tools.

In [ ]:
# Initialize MCP server
mcp_server = OChemMCPServer()

# List available tools
tools = mcp_server.list_tools()
print(f"Available MCP Tools ({len(tools)}):")
print("=" * 50)

for tool in tools:
    print(f"\n{tool['name']}:")
    print(f"  {tool['description']}")

In [ ]:
# List available prompts
prompts = mcp_server.list_prompts()
print(f"\nAvailable MCP Prompts ({len(prompts)}):")
print("=" * 50)

for prompt in prompts:
    print(f"\n{prompt['name']}:")
    print(f"  {prompt['description']}")

## 2. Using MCP Tools

Demonstrate how to call MCP tools directly.

In [ ]:
# Example 1: Predict properties
molecule = "CC(C)CC(C)(C)c1ccc(cc1)C(=O)O"  # A drug-like molecule

# Visualize molecule
mol = Chem.MolFromSmiles(molecule)
display(Draw.MolToImage(mol, size=(400, 300)))

# Call prediction tool
result = await mcp_server.handle_tool_call(
    'predict_properties',
    {
        'smiles': molecule,
        'properties': ['logP', 'MW', 'TPSA', 'QED']
    }
)

print("\nProperty Predictions:")
if 'predictions' in result:
    for prop, data in result['predictions'].items():
        print(f"{prop}: {data['value']:.2f} ± {data['uncertainty']:.2f}")

In [ ]:
# Example 2: ADMET predictions
admet_result = await mcp_server.handle_tool_call(
    'predict_admet',
    {'smiles': molecule}
)

if 'admet_properties' in admet_result:
    print("\nADMET Predictions:")
    admet = admet_result['admet_properties']
    
    # Create a visual representation
    categories = list(admet.keys())
    values = [admet[cat].get('probability', 0) for cat in categories]
    
    import matplotlib.pyplot as plt
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(categories, values, color=['green' if v > 0.7 else 'orange' if v > 0.4 else 'red' for v in values])
    ax.set_ylim(0, 1)
    ax.set_ylabel('Probability')
    ax.set_title('ADMET Profile')
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{value:.2f}', ha='center', va='bottom')
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 3. Structure Optimization via MCP

Use MCP to optimize molecular structures.

In [ ]:
# Optimize for better ADMET properties
optimization_result = await mcp_server.handle_tool_call(
    'optimize_structure',
    {
        'lead_smiles': molecule,
        'optimization_goals': {
            'logP': [2.0, 4.0],
            'MW': [300, 450],
            'QED': 0.8
        },
        'maintain_scaffold': True
    }
)

# Display results
if 'optimized_molecules' in optimization_result:
    print(f"Generated {len(optimization_result['optimized_molecules'])} optimized molecules")
    
    # Show top 3
    top_mols = optimization_result['optimized_molecules'][:3]
    
    mols_to_draw = []
    labels = []
    
    for i, opt_mol in enumerate(top_mols):
        mol = Chem.MolFromSmiles(opt_mol['smiles'])
        if mol:
            mols_to_draw.append(mol)
            labels.append(f"Score: {opt_mol['score']:.2f}\nQED: {opt_mol['properties']['QED']:.2f}")
    
    if mols_to_draw:
        img = Draw.MolsToGridImage(mols_to_draw, molsPerRow=3, 
                                 subImgSize=(350, 300), legends=labels)
        display(img)

## 4. Synthesis Planning via MCP

Use MCP for retrosynthetic analysis.

In [ ]:
# Plan synthesis for an optimized molecule
if 'optimized_molecules' in optimization_result and optimization_result['optimized_molecules']:
    target_mol = optimization_result['optimized_molecules'][0]['smiles']
    
    synthesis_result = await mcp_server.handle_tool_call(
        'suggest_synthesis',
        {
            'target_smiles': target_mol,
            'max_steps': 3
        }
    )
    
    if 'routes' in synthesis_result:
        print(f"Found {len(synthesis_result['routes'])} synthetic routes")
        
        # Display best route
        if synthesis_result['routes']:
            best_route = synthesis_result['routes'][0]
            print(f"\nBest route (score: {best_route['score']:.2f}):")
            
            for i, step in enumerate(best_route['steps']):
                print(f"\nStep {i+1}: {step['reaction_type']}")
                print(f"  Reactants: {', '.join(step['reactants'])}")
                print(f"  → Products: {', '.join(step['products'])}")

## 5. Reaction Feasibility Checking

Check if specific reactions are feasible.

In [ ]:
# Check a specific reaction
reaction_check = await mcp_server.handle_tool_call(
    'check_reaction_feasibility',
    {
        'reactants': ['c1ccccc1O', 'CC(=O)Cl'],  # Phenol + Acetyl chloride
        'products': ['CC(=O)Oc1ccccc1'],  # Phenyl acetate
        'conditions': {
            'solvent': 'DCM',
            'base': 'pyridine',
            'temperature': '0°C'
        }
    }
)

if reaction_check:
    print("Reaction Feasibility Analysis:")
    print(f"Feasible: {reaction_check.get('feasible', 'Unknown')}")
    print(f"Confidence: {reaction_check.get('confidence', 0):.2f}")
    print(f"Reaction Type: {reaction_check.get('reaction_type', 'Unknown')}")
    
    if reaction_check.get('mechanism'):
        print(f"\nMechanism: {reaction_check['mechanism'].get('type', 'Unknown')}")
        if reaction_check['mechanism'].get('steps'):
            print("Steps:")
            for step in reaction_check['mechanism']['steps']:
                print(f"  - {step}")

## 6. Using MCP Prompts

Demonstrate how to use MCP prompts for AI assistance.

In [ ]:
# Get chemistry expert prompt
expert_prompt = mcp_server.get_prompt('chemistry_expert')

if expert_prompt:
    print("Chemistry Expert Prompt:")
    print("=" * 50)
    print(expert_prompt['template'][:500] + "...")
    
    # Example: Format prompt for a specific question
    question = "How can I improve the solubility of this molecule while maintaining its activity?"
    formatted_prompt = expert_prompt['template'].format(
        molecule=molecule,
        question=question
    )
    
    print("\n\nFormatted prompt for AI:")
    print("=" * 50)
    print(formatted_prompt[:300] + "...")

## 7. Complex Workflow Example

Demonstrate a complete drug discovery workflow using MCP.

In [ ]:
async def drug_discovery_workflow(lead_smiles, target_profile):
    """Complete drug discovery workflow using MCP."""
    workflow_results = {}
    
    print("Starting Drug Discovery Workflow...")
    print("=" * 50)
    
    # Step 1: Analyze lead compound
    print("\n1. Analyzing lead compound...")
    lead_analysis = await mcp_server.handle_tool_call(
        'predict_properties',
        {'smiles': lead_smiles, 'properties': ['logP', 'MW', 'TPSA', 'QED']}
    )
    workflow_results['lead_analysis'] = lead_analysis
    
    # Step 2: Check ADMET
    print("\n2. Checking ADMET properties...")
    admet = await mcp_server.handle_tool_call(
        'predict_admet',
        {'smiles': lead_smiles}
    )
    workflow_results['lead_admet'] = admet
    
    # Step 3: Optimize structure
    print("\n3. Optimizing structure...")
    optimization = await mcp_server.handle_tool_call(
        'optimize_structure',
        {
            'lead_smiles': lead_smiles,
            'optimization_goals': target_profile,
            'maintain_scaffold': True
        }
    )
    workflow_results['optimization'] = optimization
    
    # Step 4: Select best candidate
    if optimization.get('optimized_molecules'):
        best_candidate = optimization['optimized_molecules'][0]
        
        # Step 5: Plan synthesis
        print("\n4. Planning synthesis for best candidate...")
        synthesis = await mcp_server.handle_tool_call(
            'suggest_synthesis',
            {
                'target_smiles': best_candidate['smiles'],
                'max_steps': 4
            }
        )
        workflow_results['synthesis'] = synthesis
        
        # Step 6: Check key reactions
        if synthesis.get('routes') and synthesis['routes']:
            key_step = synthesis['routes'][0]['steps'][0]
            
            print("\n5. Checking feasibility of key reaction...")
            feasibility = await mcp_server.handle_tool_call(
                'check_reaction_feasibility',
                {
                    'reactants': key_step['reactants'],
                    'products': key_step['products']
                }
            )
            workflow_results['key_reaction_check'] = feasibility
    
    return workflow_results

# Run workflow
lead = "c1ccc(cc1)C(=O)N"  # Benzamide
target_profile = {
    'logP': [2.0, 3.5],
    'MW': [250, 350],
    'QED': 0.7,
    'HBD': [1, 3]
}

results = await drug_discovery_workflow(lead, target_profile)

In [ ]:
# Visualize workflow results
print("\n\nWorkflow Results Summary:")
print("=" * 50)

# Lead properties
if 'lead_analysis' in results and 'predictions' in results['lead_analysis']:
    print("\nLead Properties:")
    for prop, data in results['lead_analysis']['predictions'].items():
        print(f"  {prop}: {data['value']:.2f}")

# ADMET issues
if 'lead_admet' in results and 'admet_properties' in results['lead_admet']:
    print("\nADMET Concerns:")
    for cat, data in results['lead_admet']['admet_properties'].items():
        if data.get('probability', 1) < 0.7:
            print(f"  ⚠️  {cat}: {data['probability']:.2f}")

# Optimization success
if 'optimization' in results and 'optimized_molecules' in results['optimization']:
    print(f"\nOptimization: Generated {len(results['optimization']['optimized_molecules'])} candidates")
    best = results['optimization']['optimized_molecules'][0]
    print(f"  Best candidate score: {best['score']:.2f}")
    print(f"  Best candidate QED: {best['properties']['QED']:.2f}")

# Synthesis feasibility
if 'synthesis' in results and 'routes' in results['synthesis']:
    print(f"\nSynthesis: Found {len(results['synthesis']['routes'])} routes")
    if results['synthesis']['routes']:
        print(f"  Best route: {results['synthesis']['routes'][0]['num_steps']} steps")

# Reaction check
if 'key_reaction_check' in results:
    print(f"\nKey Reaction Feasibility: {results['key_reaction_check'].get('feasible', 'Unknown')}")

## 8. Batch Processing

Process multiple molecules using MCP tools.

In [ ]:
# Batch process multiple molecules
test_molecules = [
    "CC(=O)Oc1ccccc1C(=O)O",  # Aspirin
    "CC(C)Cc1ccc(cc1)C(C)C(=O)O",  # Ibuprofen
    "CC(=O)Nc1ccc(O)cc1",  # Acetaminophen
    "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"  # Caffeine
]

batch_results = []

for smiles in test_molecules:
    # Get properties and ADMET
    props = await mcp_server.handle_tool_call(
        'predict_properties',
        {'smiles': smiles, 'properties': ['MW', 'logP', 'QED']}
    )
    
    admet = await mcp_server.handle_tool_call(
        'predict_admet',
        {'smiles': smiles}
    )
    
    if 'predictions' in props and 'admet_properties' in admet:
        batch_results.append({
            'SMILES': smiles,
            'MW': props['predictions']['MW']['value'],
            'logP': props['predictions']['logP']['value'],
            'QED': props['predictions']['QED']['value'],
            'Absorption': admet['admet_properties']['absorption']['probability'],
            'BBB': admet['admet_properties']['distribution'].get('bbb_penetration', 0)
        })

# Display results
df_batch = pd.DataFrame(batch_results)
df_batch = df_batch.round(2)

print("Batch Processing Results:")
display(df_batch)

## 9. MCP Server Info

Display MCP server information and capabilities.

In [ ]:
# Get server info
print("MCP Server Information")
print("=" * 50)
print(f"Server Name: {mcp_server.__class__.__name__}")
print(f"Total Tools: {len(mcp_server.list_tools())}")
print(f"Total Prompts: {len(mcp_server.list_prompts())}")

# Tool categories
tool_categories = {
    'prediction': ['predict_properties', 'predict_admet'],
    'optimization': ['optimize_structure', 'generate_analogs'],
    'synthesis': ['suggest_synthesis', 'check_reaction_feasibility'],
    'generation': ['generate_molecules']
}

print("\nTool Categories:")
for category, tools in tool_categories.items():
    available = [t for t in tools if any(tool['name'] == t for tool in mcp_server.list_tools())]
    print(f"  {category.capitalize()}: {len(available)} tools")

## Summary

In this notebook, we demonstrated:

1. **MCP Server Setup**: Initializing and exploring available tools
2. **Property Prediction**: Using MCP for molecular property and ADMET prediction
3. **Structure Optimization**: Optimizing molecules via MCP tools
4. **Synthesis Planning**: Retrosynthetic analysis through MCP
5. **Reaction Checking**: Evaluating reaction feasibility
6. **Prompt Templates**: Using MCP prompts for AI assistance
7. **Complex Workflows**: Building multi-step drug discovery pipelines
8. **Batch Processing**: Handling multiple molecules efficiently

Key benefits of MCP integration:
- **Standardized Interface**: Consistent tool calling across different AI systems
- **Modular Design**: Easy to add new chemistry tools
- **AI-Ready**: Designed for integration with Claude, GPT, and other LLMs
- **Comprehensive**: Covers prediction, optimization, and synthesis planning

Next steps:
- Integrate MCP server with your AI assistant (Claude, ChatGPT, etc.)
- Extend with custom chemistry tools
- Build domain-specific prompts
- Create automated workflows for your research